# University Speech Collection

By Guo (Jerry) Cheng, November 2024

In [4]:
import requests
import csv

## 1. Collect Speeches from YouTube (Subtitles)

In [5]:
# Set the api base url
api_base_url = "https://youtube-media-downloader.p.rapidapi.com/v2/video/"

# Set the header
headers = {
    'x-rapidapi-host': "youtube-media-downloader.p.rapidapi.com",
    'x-rapidapi-key': "d209386858msh12ca2f4db9f354dp143b66jsn918c91b7bf9b"
}

# f47bcb4bdcmsh40a27ca49e401e5p1ea55bjsnbfd8138449ac  (jerryguo000111)
# 169f17885cmsh845ebedc22a7cd6p1c3065jsn4176dccea72e  (GitHub)
# d209386858msh12ca2f4db9f354dp143b66jsn918c91b7bf9b  (chengguojerry)

# Define a function to get the video id from url
def get_youtube_id(video_url):

    # Split YouTube URL by '?v=' so we can get its ID
    items = str(video_url).split('?v=')

    # Get the last part of the list which is the ID
    video_id = items[-1]

    return video_id

# Define a function to get the video details
def get_youtube_details(video_id):

    # Send a get request to the API 
    querystring = {"videoId": video_id}
    details = requests.request("GET", 
                                api_base_url + "details", 
                                headers = headers, 
                                params = querystring)
    
    # Return json
    return details.json()

# Define a function to get youtube subtitles
def get_youtube_subtitle(subtitle_url):

    # Send a get subtitle text request to the API 
    querystring = {"subtitleUrl": subtitle_url}
    subtitle = requests.request("GET", 
                                api_base_url + "subtitles", 
                                headers = headers,
                                params = querystring)

    # Return the subtitle text
    return subtitle.text

# Define a function to gather info
def gather_speech_info(speech_type, details, subtitle):

    # Fill in all fields
    fields = [details["publishedTimeText"],                # published_date
            speech_type,                                   # speech_type
            details["type"],                               # src_type
            details["id"],                                 # src_id
            details["title"].replace("\n", '    '),        # title
            details["description"].replace("\n", '    '),  # desc
            details["channel"]["name"],                    # channel_name
            details["subtitles"]['items'][0]['url'],       # speech_url
            details["subtitles"]['items'][0]['code'],      # speech_lang
            subtitle.replace("\n", '    ')                 # speech_raw
            ]
    
    # Return fields
    return fields
    
# Define a function to write a list (append) to csv
def list2csv(file_name, list):

    # Append list to csv
    with open(file_name, "a", newline = '') as f:
        writer = csv.writer(f, quoting = csv.QUOTE_MINIMAL)
        writer.writerow(list)

    # Close file
    f.close()
    
# Define a function as a whole pipeline
def youtube2csv(speech_type, video_url, file_name):

    # Get video id
    video_id = get_youtube_id(video_url)

    # Get video details
    details = get_youtube_details(video_id)

    # Get subtitle url
    subtitle_url = details["subtitles"]['items'][0]['url']

    # Get video subtitle
    subtitle = get_youtube_subtitle(subtitle_url)

    # Get the gathered list
    fields = gather_speech_info(speech_type, details, subtitle)

    # Write to csv
    list2csv(file_name, fields)

In [6]:
# # Set attrs
# attrs = ["published_date",
#          "speech_type",
#          "src_type",
#          "src_id",
#          "title",
#          "desc",
#          "channel_name",
#          "speech_url",
#          "speech_lang",
#          "speech_raw"]

# # Create an empty csv
# list2csv("../data/university-speech.csv", attrs)

In [30]:
# Start collecting speechs
youtube2csv("commencement", 
            "https://www.youtube.com/watch?v=xk57tmbFKsQ", 
            "../data/university-speech.csv")